In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import joblib
import gc
import time

In [2]:
df = pd.read_hdf("./dataset.h5", key="df")
df

,Cens_C,Cens_Db,Cens_D,Cens_Eb,Cens_E,Cens_F,Cens_Gb,Cens_G,Cens_Ab,Cens_A,Cens_Bb,Cens_B,chord,filename
0,0.458163,0.228498,0.335618,0.385245,0.280831,0.562060,0.101404,0.142517,0.211683,0.000000,0.081103,0.007991,FMin7,../Datasets/1.mp3
1,0.456882,0.228481,0.329903,0.386718,0.270461,0.563033,0.104248,0.142325,0.223759,0.000000,0.096144,0.008832,FMin7,../Datasets/1.mp3
2,0.454308,0.229081,0.324611,0.388377,0.260669,0.563478,0.107217,0.142526,0.235052,0.000000,0.110957,0.009594,FMin7,../Datasets/1.mp3
3,0.450574,0.230340,0.319815,0.390179,0.251621,0.563336,0.110333,0.143008,0.245606,0.000000,0.125269,0.010269,FMin7,../Datasets/1.mp3
4,0.445894,0.232287,0.315578,0.392097,0.243367,0.562581,0.113524,0.143594,0.255459,0.000000,0.138864,0.010856,FMin7,../Datasets/1.mp3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0.207725,0.076254,0.397630,0.012440,0.081124,0.025550,0.113143,0.602074,0.253705,0.415162,0.012520,0.416366,AMin7,../Datasets/9.mp3
109,0.200846,0.072857,0.392787,0.013882,0.080756,0.028333,0.115661,0.609691,0.257939,0.416193,0.013882,0.409283,AMin7,../Datasets/9.mp3
110,0.194574,0.068975,0.387773,0.015407,0.080133,0.031256,0.117944,0.616832,0.261322,0.417602,0.015407,0.402578,AMin7,../Datasets/9.mp3
111,0.188929,0.064633,0.382609,0.017022,0.079286,0.034334,0.119918,0.623479,0.263771,0.419261,0.017022,0.396526,AMin7,../Datasets/9.mp3


In [3]:
unique_file = df["filename"].unique()
train_file, test_file = sk.model_selection.train_test_split(
    unique_file,
    random_state=42,
    # test_size=0.1,
    test_size=0.2,
)
# train_file, val_file = sk.model_selection.train_test_split(
#     train_file,
#     random_state=42,
#     test_size=0.1,
# )

df_train = df[df["filename"].isin(train_file)]
# df_val = df[df["filename"].isin(val_file)]
df_test = df[df["filename"].isin(test_file)]

del df
gc.collect()

0

In [4]:
print("Chords")
print("\n## TRAIN ##\n", df_train["chord"].value_counts())
# print("\n## VAL ##", df_val["chord"].value_counts())
print("\n## TEST ##\n", df_test["chord"].value_counts())

Chords

## TRAIN ##
 chord
DMaj7     12277
GMaj7     11858
AMaj7     11804
EMaj7     11803
AbMaj7    11793
BbMaj7    11765
CMin7     11718
EbMaj7    11690
BMaj7     11616
GbMaj7    11557
FMin7     11496
AbMin7    11444
CMaj7     11420
DbMaj7    11374
FMaj7     11368
EMin7     11323
GbMin7    11210
DbMin7    11114
DMin7     10945
BbMin7    10941
GMin7     10869
AMin7     10850
EbMin7    10765
BMin7     10700
G7         3035
Bb7        2928
C7         2922
B7         2798
Ab7        2762
Db7        2756
D7         2749
A7         2734
Eb7        2732
Gb7        2709
E7         2695
F7         2680
Name: count, dtype: int64

## TEST ##
 chord
EMin7     4380
DbMin7    4312
AbMin7    4306
GbMin7    4302
BMin7     4276
AMin7     4180
CMin7     4151
FMin7     4102
EbMin7    4031
GMin7     4016
BbMin7    4013
DMin7     4006
GMaj7     3446
AMaj7     3407
CMaj7     3239
DMaj7     3187
EMaj7     3116
GbMaj7    3102
EbMaj7    3097
BbMaj7    3097
DbMaj7    3094
FMaj7     3076
AbMaj7    3076
BMaj7  

In [5]:
# y = df["chord"]
# X = df.drop(columns="chord")
# encoder = sk.preprocessing.LabelEncoder()
# y_encoded = encoder.fit_transform(y)

# joblib.dump(encoder, "./encoder.xz")

def get_Xy(df: pd.DataFrame):
    return df.drop(["chord", "filename"], axis=1), df["chord"]

X_train, y_train = get_Xy(df_train)
# X_val, y_val = get_Xy(df_val)
X_test, y_test = get_Xy(df_test)

encoder = sk.preprocessing.LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
# y_val_encoded = encoder.transform(y_val)
y_test_encoded = encoder.transform(y_test)

joblib.dump(encoder, "./encoder.xz")

['./encoder.xz']

In [6]:
# SEQUENCE_LEN = 20 # 0.1 * 20.0 = 2 sec of sequence data

# def make_seq(X, y_encoded):
#     X_seq, y_encoded_seq = None, None
#     X_seq_list = []
#     y_encoded_seq_list = []
#     for i in range(len(X) - SEQUENCE_LEN + 1):
#         X_seq_list.append(X.values[i : i + SEQUENCE_LEN, :].flatten())
#         y_encoded_seq_list.append(y_encoded[i + SEQUENCE_LEN - 1])
    
#     return np.array(X_seq_list), np.array(y_encoded_seq_list)

# X_train, y_train_encoded = make_seq(X_train, y_train_encoded)
# # X_val, y_val_encoded = make_seq(X_val, y_val_encoded)
# X_test, y_test_encoded = make_seq(X_test, y_test_encoded)

# print("TRAIN")
# print("X sequence shape: ", X_train.shape)
# print("y sequence shape: ", y_train_encoded.shape)

# # print("\nVAL")
# # print("X sequence shape: ", X_val.shape)
# # print("y sequence shape: ", y_val_encoded.shape)

# print("\nTEST")
# print("X sequence shape: ", X_test.shape)
# print("y sequence shape: ", y_test_encoded.shape)

# gc.collect()

# Models

## SVM

In [7]:
seq_lens = [5, 10, 20]

def make_seq(X, y_encoded, seq_len):
    X_seq, y_encoded_seq = None, None
    X_seq_list = []
    y_encoded_seq_list = []
    # for i in range(len(X) - SEQUENCE_LEN + 1):
    #     X_seq_list.append(X.values[i : i + SEQUENCE_LEN, :].flatten())
    #     y_encoded_seq_list.append(y_encoded[i + SEQUENCE_LEN - 1])
    for i in range(len(X) - seq_len + 1):
        X_seq_list.append(X.values[i : i + seq_len, :].flatten())
        y_encoded_seq_list.append(y_encoded[i + seq_len - 1])
    
    return np.array(X_seq_list), np.array(y_encoded_seq_list)

for seq_len in seq_lens: 
    X_train_seq, y_train_encoded_seq = make_seq(X_train, y_train_encoded, seq_len)
    # X_val, y_val_encoded = make_seq(X_val, y_val_encoded)
    X_test_seq, y_test_encoded_seq = make_seq(X_test, y_test_encoded, seq_len)

    print(f'\n:::: SVM SEQ {seq_len} ::::')
    model_svc = sk.svm.SVC(
        # class_weight=class_weight_dict,
        class_weight="balanced",
        # C=1000,
        # gamma=0.001,
        # verbose=1
    )
    
    start = time.time()
    model_svc.fit(
        X_train_seq,
        y_train_encoded_seq,
    )
    end = time.time()
    print(f"Training duration: {end - start}sec")

    start = time.time()
    y_pred = model_svc.predict(X_test_seq)
    print(f"accuracy: {sk.metrics.accuracy_score(y_pred, y_test_encoded_seq) * 100}%")
    end = time.time()
    print(f"Testing duration: {end - start}sec")
    
    print(
        sk.metrics.classification_report(y_test_encoded_seq, y_pred, target_names=encoder.classes_)
    )
    
    joblib.dump(model_svc, f"./model_svm_seq_{seq_len}.xz")
    
    gc.collect()


:::: SVM SEQ 5 ::::
Training duration: 1127.2085177898407sec
accuracy: 59.50202813497886%
Testing duration: 1285.8626244068146sec
              precision    recall  f1-score   support

          A7       0.25      0.57      0.35       372
       AMaj7       0.58      0.54      0.56      3407
       AMin7       0.64      0.60      0.62      4180
         Ab7       0.24      0.59      0.34       386
      AbMaj7       0.57      0.58      0.57      3076
      AbMin7       0.66      0.61      0.64      4306
          B7       0.40      0.62      0.49       460
       BMaj7       0.56      0.52      0.54      3038
       BMin7       0.65      0.61      0.63      4276
         Bb7       0.36      0.59      0.45       383
      BbMaj7       0.60      0.56      0.58      3097
      BbMin7       0.63      0.57      0.60      4013
          C7       0.33      0.63      0.44       399
       CMaj7       0.57      0.56      0.57      3235
       CMin7       0.65      0.65      0.65      4151
    

In [8]:

# grid_search_svc.fit(X_train, y_train_encoded)